In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/Othercomputers/My laptop/ML/YAGPT')

In [3]:
from config import params
from pprint import pprint
onServer = True

if not onServer:
    params['num_proc'] = 1
pprint(params)

{'additional_special_tokens': ['speaker1: ',
                               'speaker2: ',
                               'speaker3: ',
                               'speaker4: ',
                               'speaker5: ',
                               'speaker6: ',
                               'speaker7: ',
                               'speaker8: ',
                               'speaker9: ',
                               'history: '],
 'checkpoint': 'AlexWortega/instruct_rugptMedium',
 'data_path': 'encoded_texts.pkl',
 'num_proc': 5}


In [7]:
!pip install transformers datasets huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 52.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 19.6 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM


checkpoint = params['checkpoint']
tokenizer = AutoTokenizer.from_pretrained(checkpoint, pad_token='<|pad|>')
n = 10
special_tokens = {'additional_special_tokens': params['additional_special_tokens']}
tokenizer.add_special_tokens(special_tokens)

model = AutoModelForCausalLM.from_pretrained(checkpoint)
model.resize_token_embeddings(len(tokenizer))

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Embedding(50274, 1024)

In [ ]:
params

{'data_path': 'encoded_texts.pkl',
 'checkpoint': 'AlexWortega/instruct_rugptMedium',
 'additional_special_tokens': ['speaker1: ',
  'speaker2: ',
  'speaker3: ',
  'speaker4: ',
  'speaker5: ',
  'speaker6: ',
  'speaker7: ',
  'speaker8: ',
  'speaker9: ',
  'history: '],
 'num_proc': 5}

In [11]:
import pickle

data_path = params['data_path']
with open(data_path, 'rb') as f:
    data = pickle.load(f)

FileNotFoundError: ignored

In [ ]:
from pprint import pprint
pprint(data[0].keys())
print(len(data))

In [ ]:
type(data)

In [ ]:
# We need to add labels to our data
def add_labels(samples):
    samples['labels'] = samples['input_ids'].copy()
    return samples

data.map(add_labels, batched=True, num_proc=params['num_proc'])

In [ ]:
from datasets import train_test_split

data.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
data_dict = data.train_test_split(test_size=0.1, shuffle=True)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch' # evaluation strategy to adopt during training
)

trainer = Trainer(
    model=model,                         
    args=training_args,
    train_dataset=data['train'],
    eval_dataset=data['test']
)

In [ ]:
trainer.train()

In [ ]:
import math

eval_result = trainer.evaluate() # returns a dict with the evaluation metrics, cross-entropy loss and perplexity
pprint(eval_result)